In [1]:
import pandas as pd

In [4]:
#mapping=pd.read_csv('wd2schema-std.csv', index_col=0)
mapping=pd.read_csv('../wd_class_name_mapping.csv', index_col=0)

In [ ]:
mapping

,sch_id,wd_id,sch_cname,wd_cname
0,http://schema.org/AccountingService,http://www.wikidata.org/wiki/accounting_services,AccountingService,accounting_services
1,http://schema.org/Action,http://www.wikidata.org/wiki/action,Action,action
2,http://schema.org/Airline,http://www.wikidata.org/wiki/airline,Airline,airline
3,http://schema.org/Airport,http://www.wikidata.org/wiki/airport,Airport,airport
4,http://schema.org/AmusementPark,http://www.wikidata.org/wiki/amusement_park,AmusementPark,amusement_park
...,...,...,...,...
299,http://schema.org/Winery,http://www.wikidata.org/wiki/winery,Winery,winery
300,http://schema.org/WorkersUnion,http://www.wikidata.org/wiki/labor_union,WorkersUnion,labor_union
301,http://schema.org/Zoo,http://www.wikidata.org/wiki/zoo,Zoo,zoo
302,http://schema.org/FALSE,http://www.wikidata.org/wiki/false,FALSE,false


In [ ]:
mapping['wd_id']=["http://www.wikidata.org/wiki/"+i for i in mapping['wd_id']]
mapping['sch_id']=["http://schema.org/"+i for i in mapping['sch_id']]
mapping=mapping[['sch_id','wd_id']]

In [26]:
from SPARQLWrapper import SPARQLWrapper, JSON
def getWDLabel(uri):
    sparq1=SPARQLWrapper("https://query.wikidata.org/sparql")
    queryString= """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
        PREFIX wd: <http://www.wikidata.org/entity/> 
        SELECT  *
        WHERE {
        %s rdfs:label ?label .
        FILTER (langMatches( lang(?label), "EN" ) )
      } 
      limit 1
      """%(uri)
    sparq1.setQuery (queryString)
    sparq1.setReturnFormat(JSON)
    results=sparq1.query().convert()
    for r in results["results"]["bindings"]:
        return(r["label"]["value"])
    

In [28]:
def GetClassLabel(URIString):
    if len(URIString.split(':')) > 2:
        Label = URIString.split(':')[-1]
    else:
        Label = URIString.split('/')[-1]
    return Label
    

In [59]:
import csv

ClassList=list(mapping['wd_id'])

sparq1=SPARQLWrapper("https://wikidata.demo.openlinksw.com/sparql")

with open('WikiData_instances.csv', 'w') as f:
        writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        line= ['Class_Name', 'Class_URI' ,'Instance_Name', 'Instance_URI']
        writer.writerow(line)
        for className in ClassList:
            MyList=[]
            try:
                queryString= """
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:<http://www.w3.org/2002/07/owl#>
PREFIX wd: <http://www.wikidata.org/entity/> 

SELECT ?label
WHERE {
 ?name wdt:P31 wd:%s .
 ?name rdfs:label ?label .
 FILTER (langMatches( lang(?label), "EN" ) )
 
 }
                            """%(GetClassLabel(className))
                sparq1.setQuery (queryString)
                sparq1.setReturnFormat(JSON)
                results=sparq1.query().convert()
                
            except TypeError:
                continue

            if results== None:
                continue
            else:
                for r in results["results"]["bindings"]:
                    MyList.append(r["label"]["value"])

                for i in range(len(MyList)):
                    MyList[i]=  MyList[i].replace(' ','_')
                    line=[]
                    line.append(mapping[mapping['wd_id']==className]['wd_cname'].values[0])
                    line.append(mapping[mapping['wd_id']==className]['wd_cname'].values[0])#className
                    line.append(GetClassLabel(MyList[i]))
                    line.append(MyList[i])
                    writer.writerow(line)

In [67]:
queryString= """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:<http://www.w3.org/2002/07/owl#>
PREFIX wd: <http://www.wikidata.org/entity/> 

SELECT ?label
WHERE {
?name wdt:P31 wd:Q46970.
?name rdfs:label ?label .
FILTER (langMatches( lang(?label), "EN" ) )

}
            """
sparq1.setQuery (queryString)
sparq1.setReturnFormat(JSON)
results=sparq1.query().convert()

In [ ]:
df['Instance_URI']=['http://www.wikidata.org/wiki/'+str(i) for i in df['Instance_URI']]
df['Class_URI']=['http://www.wikidata.org/wiki/'+str(i) for i in df['Class_URI']]

In [ ]:
df=pd.read_csv('WikiData_instances.csv',index_col=0)
#df.Class_Name.unique()
#df.Class_Name=[str(x).replace('','_') for x in df.Class_Name ]
df.Class_Name = df.Class_Name.replace(' ', '_')

In [ ]:
df['Class_Name']=[str(i).replace(' ','_') for i in df['Class_Name']]